In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [5]:
!#pip install tensorflow


In [6]:
import tensorflow as tf
print(tf.__version__)
tf.config.list_physical_devices()

2.20.0


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [7]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [9]:
data = pd.read_csv("Iris.csv", index_col = "Id")
data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa


In [10]:
X = data.drop("Species", axis=1).to_numpy()
y = data[["Species"]].to_numpy()


In [11]:
scaler = StandardScaler()
onehot_encoder = OneHotEncoder(sparse_output=False)

enc_y = onehot_encoder.fit_transform(y)
scaled_X = scaler.fit_transform(X)


In [12]:
cat_array = onehot_encoder.categories_[0]
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, enc_y, test_size=0.2, shuffle=True, stratify=enc_y, random_state=999
)

### Model Creation and Compile

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

In [14]:
inp_shape = X_train.shape[1]
num_classes = y_train.shape[1]

In [15]:
model = Sequential()

model.add(Input(shape= (inp_shape,))) #< Input Layer

model.add(Dense(64, activation="relu")) #< Hidden Layer
model.add(Dense(32, activation="relu")) #< Hidden Layer

model.add(Dense(num_classes, activation="softmax")) #< Output Layer

model.compile(
    optimizer="Adam", loss="categorical_crossentropy" , metrics=["accuracy"]
)

In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,499 (9.76 KB)

 Trainable params: 2,499 (9.76 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
history = model.fit(
    X_train, y_train, epochs=25, batch_size=32, verbose=1
)

Epoch 1/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2750 - loss: 1.0929  
Epoch 2/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5583 - loss: 1.0006 
Epoch 3/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6083 - loss: 0.9203 
Epoch 4/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6583 - loss: 0.8469 
Epoch 5/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6583 - loss: 0.7810 
Epoch 6/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6667 - loss: 0.7202 
Epoch 7/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6667 - loss: 0.6684 
Epoch 8/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6667 - loss: 0.6190 
Epoch 9/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6667 - loss: 0.5787 
Epoch 10/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6667 - loss: 0.5447 
Epoch 11/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6917 - loss: 0.5146 
Epoch 12/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7250 - loss: 0.4892 


In [18]:
loss, accuracy = model.evaluate(X_test, y_test)
accuracy

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.9333 - loss: 0.2841


0.9333333373069763

In [19]:
y_pred = np.argmax(model.predict(X_test), axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


In [20]:
y_true = np.argmax(y_test, axis=1)

In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score


In [22]:
confusion_matrix(y_true, y_pred)

array([[10,  0,  0],
       [ 0,  8,  2],
       [ 0,  0, 10]])

In [23]:
accuracy_score(y_true, y_pred)

0.9333333333333333

In [24]:
f1_score(y_true, y_pred, average="macro")

0.9326599326599326

In [25]:
precision_score(y_true, y_pred, average="macro")

0.9444444444444445

In [26]:
recall_score(y_true, y_pred, average="macro")

0.9333333333333332